In [41]:
#!/usr/bin/env python

import numpy as np
import cv2
from numpy.core.numeric import full
import rospy
import rospkg
import open3d as o3d
from transforms3d.quaternions import mat2quat
import time
import copy
import os
from copy import deepcopy

import sys
from os import path
from ocrtoc_common.camera_interface import CameraInterface
from ocrtoc_common.transform_interface import TransformInterface
from sensor_msgs.msg import CameraInfo, Image, PointCloud2
import pickle


if __name__ == "__main__" :
    rospack = rospkg.RosPack()
    taskid= '3-1-1-data-2'
    save_path = os.path.join(rospack.get_path('ocrtoc_perception'),'data',taskid, '')
    full_pcd = o3d.io.read_point_cloud(save_path + 'full_pcd.pcd') 
    with open(save_path +  'color_images.pickle', 'rb') as handle:
        color_images = pickle.load(handle)  
    with open(save_path + 'camera_poses.pickle', 'rb') as handle:
        camera_poses = pickle.load(handle) 
    # Compute Grasping Poses (Many Poses in a Scene)
    with open(save_path + 'gg.pickle', 'rb') as handle:
        gg = pickle.load(handle)      

    with open(save_path + 'object_poses.pickle', 'rb') as handle:
        object_poses= pickle.load(handle) 
        
        
        
    frame = o3d.geometry.TriangleMesh.create_coordinate_frame(0.1)        


In [ ]:
#for object_ in object_poses.values():
    #print(object_["pose"])
    
object_pose_list = list(object_poses.values())
object_pose_list_len = len(object_pose_list)

object_origin_extracted = [None]*object_pose_list_len

for i in range(object_pose_list_len):
    object_pose = object_pose_list[i]['pose']
    #object_rot = mat2quat(object_pose[0:3,0:3])
    object_trans = object_pose[0:3,3]
    
    #p_stamped.pose.position.x = object_trans[0]
    #p_stamped.pose.position.y = object_trans[1]
    #p_stamped.pose.position.z = object_trans[2]

    #p_stamped.pose.orientation.x = object_rot[0]
    #p_stamped.pose.orientation.y = object_rot[1]
    #p_stamped.pose.orientation.z = object_rot[2]
    #p_stamped.pose.orientation.w = object_rot[3]

    #p_stamped.header.frame_id = 'world'

    object_origin_extracted[i] = object_trans
    
    
print(object_origin_extracted)

In [37]:
print(gg.scores)
#print(object_poses)

def assign_grasps_to_object(rs, ts, object_poses):
    dist_thresh = np.inf
    min_dists = np.inf * np.ones((len(rs)))
    min_object_ids = -1 * np.ones(shape = (len(rs)), dtype = np.int32)
    for i, object_name in enumerate(object_poses.keys()):
        #object_names.append(object_name)
        object_pose = object_poses[object_name]

        dists = np.linalg.norm(ts - object_pose['pose'][:3,3], axis=1)
        object_mask = np.logical_and(dists < min_dists, dists < dist_thresh)

        min_object_ids[object_mask] = i
        min_dists[object_mask] = dists[object_mask]
    return object_mask, min_object_ids


[0.06916754 0.09396293 0.09453522 0.3511787  0.3198529  0.07040492
 0.13586591 0.31745163 0.07259636 0.20223796 0.15350817 0.04152132
 0.19123724 0.2170125  0.54274297 0.3611803 ]


In [38]:

def center_of_mass_score(gg, object_poses):
    penalty_factor = 1
    ts = gg.translations
    rs = gg.rotation_matrices
    depths = gg.depths
    ts = ts + rs[:,:,0]*(np.vstack((depths, depths, depths)).T)
    dists = []
    object_mask, min_object_ids = assign_grasps_to_object(rs, ts, object_poses)
    print(min_object_ids)
    print(object_mask)
    for i, object_name in enumerate(object_poses.keys()):
        object_pose = object_poses[object_name]
        ts_temp = ts[min_object_ids == i]
        dists.append(np.linalg.norm(ts_temp-object_pose['pose'][:3,3], axis=1))
        
    dists = np.concatenate(dists)
    print(dists)
    print(gg.scores)
    gg.scores = gg.scores - dists
    gg.scores = gg.scores/max(gg.scores)
    return gg


gg = center_of_mass_score(gg, object_poses)
print(gg.scores)

[3 2 2 3 3 2 2 1 1 3 2 2 3 3 1 2]
[ True False False  True  True False False False False  True False False
  True  True False False]
[0.07063239 0.26835842 0.26423557 0.32806445 0.37414299 0.14514904
 0.18660861 0.42654906 0.26496067 0.20270731 0.1990458  0.23253512
 0.21055818 0.2421209  0.20822449 0.19195155]
[0.06916754 0.09396293 0.09453522 0.3511787  0.3198529  0.07040492
 0.13586591 0.31745163 0.07259636 0.20223796 0.15350817 0.04152132
 0.19123724 0.2170125  0.54274297 0.3611803 ]
[-0.00437899 -0.5213329  -0.5072974   0.06909711 -0.16229327 -0.22343798
 -0.15168877 -0.32613277 -0.5750484  -0.00140304 -0.1361289  -0.57101125
 -0.05775748 -0.07505836  1.          0.5058877 ]


In [42]:
o3d.visualization.draw_geometries([frame, full_pcd, *gg.to_open3d_geometry_list()])


In [ ]:
print(full_pcd.segment_plane)

dir(full_pcd)

In [48]:
out_np = np.asarray(full_pcd.points)
np.savetxt("punktmoln3-1-1.csv", out_np, delimiter = ",")